In [ ]:
from sketch.examples.prompt_machine import *

async def chatbot(response_callback):
    messages = []
    async with websockets.connect(PM_SETTINGS['uri'], extra_headers={"Cookie": f"Authorization=Bearer {PM_SETTINGS['BOT_TOKEN']}"}) as ws:
        while True:
            try:
                raw_data = await asyncio.wait_for(ws.recv(), timeout=0.5)
                data = json.loads(raw_data)
                print("log:", data)
                if data.get("replay", False) or data["sender"] == "datAI":
                    messages.append(data)
                    continue
                if data["message"] == "break":
                    break
                if data.get("meta", False):
                    continue
                messages.append(data)
            except asyncio.TimeoutError:
                # not sure why I'm doing this, it felt important to have an "infinite" loop...
                continue
            conversation = "\n".join([f"{m['sender']}: {m['message']}" for m in messages[-10:]])
            response = response_callback(conversation)
            await ws.send(json.dumps({"message": response}))


In [ ]:
PM_SETTINGS['uri']

In [ ]:
respond = GPT3Prompt("basic_response", """
this is a conversation between a human and a dog
{{conversation}}
dog:
""")


def basic_talker(conversation):
    response = respond(conversation)
    if len(response) > 10:
        response = "LONG RESPONSE" + response
    return response

convo = Prompt("basic_convo", basic_talker)


In [ ]:
await chatbot(convo)